# Basic Steps to do Storing


** Please refer to the code "pf_miner.py"

**  Here we are taking data and storing it in S3, so connecting to S3. 

**  This part is part of the second S - Storing, but it is inside code so we will briefly talk about it here.

**  We connect to AWS S3 with our AWS credentials and *.yaml file which stores them

**  We create a bucket in AWS, here the name is "finalprojectpitchfork." 

**  We upload our data each time we get a review using the function "upload_to_s3"---include Pitchfork review albums - this page contains links to each of the reviews

**  We fetch the links from here and then fetch the actual review page using the links
and each individual review page contains information we want. 

**  Once we check that this code is running on local machine I created an EC2 instance on AWS to run it.

# Detailed steps on how I Store the data

###### Please note, I am trying to run the code in Notebook here to make it look modular. The reason I am running into errors here is I have put some of the functions and other files in separate folders for proper book keeping. So from this location I would have to change the paths of the functions I am calling. Recommended way to run this--- All the scripts will run in PyCharm or Atom. 

In [ ]:
#!/usr/bin/spark-submit

from os.path import expanduser
import requests #when we type in a link in browser we make a request to that link
#we are using request to fetch information from link, once we get it we feed it to soup
#Soup parses html and gives us a tree and we go in there and get the information out of it 
#using soup
#This is how we are doing webscraping, as shown below. 
import json
import yaml
import pytz
from datetime import datetime
from dateutil.parser import parse as date_parse
from dateutil.relativedelta import relativedelta
from boto.s3.connection import S3Connection
from boto.s3.key import Key
from concurrent.futures import ThreadPoolExecutor, as_completed
from bs4 import BeautifulSoup
from time import sleep
from make_db import partition_job


In [ ]:
# The base url is the pitchfork reviews page and it returns review columns as a list
pf_url = 'http://pitchfork.com'
base_url = 'http://pitchfork.com/reviews/albums/'
review_columns = ['artist','album','score', 'album_art','genre','label',
                'pub_date','abstract','featured_tracks', 'review_content']


![pitchfork reviews](images/Scratch_Pitchfork.png)

In [ ]:
def connect_to_s3(credentials, bucket_name='finalprojectpitchfork'):

    conn = S3Connection(credentials['aws']['access_key_id'],credentials['aws']['secret_access_key'])

    try:
        bucket = conn.get_bucket(bucket_name)
        print("Bucket found on S3")
    except:
        bucket = conn.create_bucket(bucket_name)
        print("Creating bucket")
    return conn, bucket


def upload_to_s3(bucket, data):

    try:
        k = Key(bucket)
        k.key = data['artist'] + ' - ' + data['album']
        k.set_contents_from_string(json.dumps(data, ensure_ascii=False))
        print("Uploaded ", k.key)
    except:
        print("Failed to upload", k.key)

### Please refer to "pf_miner.py" for details and to 1_Streaming_data

** Please refer to "ANAMAI-What Mountain" file in this folder to see AWS S3 downloaded data example from Pitchfork.

** Please refer to "S3_bucket.png" and "S3_bucket_file.png" for screenshots of bucket and files under it.